In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.model_selection
import sklearn.linear_model
import sklearn.preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import csv
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


***
## **PRE PROCESSING :** ##
***

### **Premier dataset :** *"english_premier_league_data.csv" dont on conserve toutes les features statistiques* ###

In [2]:
df1 = pd.read_csv('Result_dataset/english_premier_league_data.csv', sep=';')
df1 = df1[df1['year'].isin([2016, 2017, 2018, 2019, 2020])] # on se restreint à ces années 
df1 = df1.rename(columns={'year': 'season'})
df1 = df1.drop(["Score","Half Time Score"], axis = 1)
df1 = df1.rename(columns={'Home Team Goals Scored': 'HT Goals Scored', 'Away Team Goals Scored': 'AT Goals Scored', # pour gagner de la place on remplace "HT" -> "HT" et "Away Team" -> "AT"
                          'Home Team Goals Conceeded': 'HT Goals Conceeded', 'Away Team Goals Conceeded': 'AT Goals Conceeded',
                          'Home Team Rating': 'HT Rating', 'Away Team Rating': 'AT Rating', 
                          'Home Team Possession %': 'HT Posses %', 'Away Team Possession %': 'AT Posses %',
                          'Home Team Off Target Shots': 'HT OffT Shots', 'Away Team Off Target Shots': 'AT OffT Shots',
                          'Home Team On Target Shots': 'HT OnT Shots', 'Away Team On Target Shots': 'AT OnT Shots',
                          'Home Team Total Shots': 'HT TotShots', 'Away Team Total Shots': 'AT TotShots',
                          'Home Team Blocked Shots': 'HT Block Shots', 'Away Team Blocked Shots': 'AT Block Shots',
                          'Home Team Corners': 'HT Corners', 'Away Team Corners': 'AT Corners',
                          'Home Team Throw Ins': 'HT Throw Ins', 'Away Team Throw Ins': 'AT Throw Ins',
                          'Home Team Pass Success %': 'HT Pass %', 'Away Team Pass Success %': 'AT Pass %',
                          'Home Team Aerials Won': 'HT Aerials Won', 'Away Team Aerials Won': 'AT Aerials Won',
                          'Home Team Clearances': 'HT Clear', 'Away Team Clearances': 'AT Clear',
                          'Home Team Fouls': 'HT Fouls', 'Away Team Fouls': 'AT Fouls',
                          'Home Team Second Yellow Cards': 'HT SecYellowC', 'Away Team Second Yellow Cards': 'AT SecYellowC',
                          'Home Team Red Cards': 'HT RedC', 'Away Team Red Cards': 'AT RedC',
                          'Home Team Yellow Cards': 'HT YellowC', 'Away Team Yellow Cards': 'AT YellowC',
                          })


df1.head()

,Home Team,Away Team,Match Excitement,HT Rating,AT Rating,HT Posses %,AT Posses %,HT OffT Shots,HT OnT Shots,HT TotShots,HT Block Shots,HT Corners,HT Throw Ins,HT Pass %,HT Aerials Won,HT Clear,HT Fouls,HT YellowC,HT SecYellowC,HT RedC,AT OffT Shots,AT OnT Shots,AT TotShots,AT Block Shots,AT Corners,AT Throw Ins,AT Pass %,AT Aerials Won,AT Clear,AT Fouls,AT YellowC,AT SecYellowC,AT RedC,HT Goals Scored,AT Goals Scored,HT Goals Conceeded,AT Goals Conceeded,season
760,HULL CITY,LEICESTER CITY,5.9,6.7,5.4,50,50,7.0,5.0,14.0,2.0,5.0,23.0,76.0,19.0,29.0,8.0,2.0,0.0,0.0,9.0,5.0,18.0,4.0,3.0,16.0,78.0,18.0,15.0,17.0,2.0,0.0,0.0,2,1,1,2,2016
761,EVERTON,TOTTENHAM,4.9,7.0,6.4,42,58,4.0,6.0,12.0,2.0,5.0,21.0,73.0,16.0,40.0,10.0,0.0,0.0,0.0,4.0,4.0,13.0,5.0,6.0,42.0,80.0,10.0,22.0,14.0,0.0,0.0,0.0,1,1,1,1,2016
762,MIDDLESBROUGH,STOKE,4.5,6.4,6.2,46,54,6.0,2.0,12.0,4.0,9.0,20.0,75.0,22.0,20.0,18.0,3.0,0.0,0.0,9.0,1.0,12.0,2.0,6.0,23.0,79.0,27.0,16.0,14.0,5.0,0.0,0.0,1,1,1,1,2016
763,BURNLEY,SWANSEA,4.0,5.1,7.9,47,53,5.0,3.0,10.0,2.0,7.0,41.0,71.0,20.0,16.0,10.0,3.0,0.0,0.0,6.0,9.0,17.0,2.0,4.0,13.0,78.0,32.0,49.0,14.0,2.0,0.0,0.0,0,1,1,0,2016
764,SOUTHAMPTON,WATFORD,4.6,7.0,6.1,64,36,10.0,6.0,24.0,8.0,6.0,25.0,85.0,9.0,21.0,8.0,1.0,0.0,0.0,4.0,1.0,5.0,0.0,2.0,12.0,69.0,19.0,36.0,12.0,2.0,0.0,1.0,1,1,1,1,2016


***
### **Second dataset :** *"spi_matches.csv" dont on extrait certains features* ###

In [3]:
df2 = pd.read_csv('SPI_dataset/spi_matches.csv', sep=';')

df2.dropna()

ligues = ['Barclays Premier League']
df2 = df2[df2['league'].isin(ligues)] # que la Premier League

df2 = df2[df2['season'].isin([2016, 2017, 2018, 2019, 2020])] #on se retreint à ces années
df2 = df2.rename(columns={'team1': 'Home Team', 'team2': 'Away Team', # on renomme les features comme df
                          'score1': 'HT Goals Scored', 'score2': 'AT Goals Scored', # pour gagner de la place on remplace "HT" -> "HT" et "Away Team" -> "AT"
                          'xg1': 'HT xG', 'xg2': 'AT xG',
                          'nsxg1': 'HT NSxG', 'nsxg2': 'AT NSxG',
                          'spi1': 'HT SPI Rate', 'spi2': 'AT SPI Rate',
                          'prob1': 'HT ProbWin', 'prob2': 'AT ProbWin',
                          'proj_score1': 'HT ProjScore', 'proj_score2': 'AT ProjScore',
                          'adj_score1': 'HT AdjScore', 'adj_score2': 'AT AdjScore',
                          'importance1': 'HT Importance', 'importance2': 'AT Importance',
                          })

df2 = df2.drop(['league_id', 'league'], axis = 1)


df2.head()



,season,date,Home Team,Away Team,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT Goals Scored,AT Goals Scored,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore
13,2016,2016-08-13,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
15,2016,2016-08-13,Hull,Leicester,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
17,2016,2016-08-13,Southampton,Watford,69.49,59.33,0.5759,0.1874,0.2367,1.91,1.05,34.1,30.7,1.0,1.0,1.05,0.22,1.52,0.41,1.05,1.05
18,2016,2016-08-13,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,0.2847,1.35,1.14,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05
19,2016,2016-08-13,Burnley,Swansea,58.98,59.74,0.4482,0.2663,0.2854,1.37,1.05,36.5,29.1,0.0,1.0,1.24,1.84,1.71,1.56,0.00,1.05


In [4]:
data_set_1 = df1["Away Team"].unique() #liste des équipes de df1 sans doublons
data_set_2 = df2["Away Team"].unique() #liste des équipes de df2 sans doublons

In [5]:
fuzz.WRatio("TOTTENHAM","Tottenham Hotspur") # etudie la correspondance des string


90

In [6]:
# on veut identifier les noms d'equipes dans chaque dataframe pour que le merge se fasse correctement

def create_team_dict(data_set_1, data_set_2): #cree dictionnaire avec {nom dataset2 : nom dataset1}
    team_dict = {}
    for team2 in data_set_2:
        for team1 in data_set_1:
            ratio = fuzz.WRatio(team1, team2)
            if ratio >= 87: # ratio de 87 assure la correspondance pour CE dataset
                team_dict[team2] = team1
    return team_dict


In [7]:
team_names = create_team_dict(data_set_1,data_set_2)
df2['Home Team'] = df2['Home Team'].replace(team_names) #remplace nom dataset2 par nom dataset1
df2['Away Team'] = df2['Away Team'].replace(team_names)
df2.head()

,season,date,Home Team,Away Team,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT Goals Scored,AT Goals Scored,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore
13,2016,2016-08-13,EVERTON,TOTTENHAM,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
15,2016,2016-08-13,HULL CITY,LEICESTER CITY,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
17,2016,2016-08-13,SOUTHAMPTON,WATFORD,69.49,59.33,0.5759,0.1874,0.2367,1.91,1.05,34.1,30.7,1.0,1.0,1.05,0.22,1.52,0.41,1.05,1.05
18,2016,2016-08-13,CRYSTAL PALACE,WEST BROM,55.19,58.66,0.4214,0.2939,0.2847,1.35,1.14,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05
19,2016,2016-08-13,BURNLEY,SWANSEA,58.98,59.74,0.4482,0.2663,0.2854,1.37,1.05,36.5,29.1,0.0,1.0,1.24,1.84,1.71,1.56,0.00,1.05


***
### **FUSION DES DEUX DATAFRAMES** ###

In [8]:
# on merge par le score,les couples d'équipes et l'année 
# ON A DONC NOTRE DATAFRAME PROPRE

df = pd.merge(df1, df2, on=['season', 'Home Team', 'Away Team','HT Goals Scored', 'AT Goals Scored']) 
df.head()


,Home Team,Away Team,Match Excitement,HT Rating,AT Rating,HT Posses %,AT Posses %,HT OffT Shots,HT OnT Shots,HT TotShots,HT Block Shots,HT Corners,HT Throw Ins,HT Pass %,HT Aerials Won,HT Clear,HT Fouls,HT YellowC,HT SecYellowC,HT RedC,AT OffT Shots,AT OnT Shots,AT TotShots,AT Block Shots,AT Corners,AT Throw Ins,AT Pass %,AT Aerials Won,AT Clear,AT Fouls,AT YellowC,AT SecYellowC,AT RedC,HT Goals Scored,AT Goals Scored,HT Goals Conceeded,AT Goals Conceeded,season,date,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore
0,HULL CITY,LEICESTER CITY,5.9,6.7,5.4,50,50,7.0,5.0,14.0,2.0,5.0,23.0,76.0,19.0,29.0,8.0,2.0,0.0,0.0,9.0,5.0,18.0,4.0,3.0,16.0,78.0,18.0,15.0,17.0,2.0,0.0,0.0,2,1,1,2,2016,2016-08-13,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,38.1,22.2,0.85,2.77,0.17,1.25,2.10,1.05
1,EVERTON,TOTTENHAM,4.9,7.0,6.4,42,58,4.0,6.0,12.0,2.0,5.0,21.0,73.0,16.0,40.0,10.0,0.0,0.0,0.0,4.0,4.0,13.0,5.0,6.0,42.0,80.0,10.0,22.0,14.0,0.0,0.0,0.0,1,1,1,1,2016,2016-08-13,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,31.9,48.0,0.73,1.11,0.88,1.81,1.05,1.05
2,MIDDLESBROUGH,STOKE,4.5,6.4,6.2,46,54,6.0,2.0,12.0,4.0,9.0,20.0,75.0,22.0,20.0,18.0,3.0,0.0,0.0,9.0,1.0,12.0,2.0,6.0,23.0,79.0,27.0,16.0,14.0,5.0,0.0,0.0,1,1,1,1,2016,2016-08-13,56.32,60.35,0.4380,0.2692,0.2927,1.30,1.01,33.9,32.5,1.40,0.55,1.13,1.06,1.05,1.05
3,BURNLEY,SWANSEA,4.0,5.1,7.9,47,53,5.0,3.0,10.0,2.0,7.0,41.0,71.0,20.0,16.0,10.0,3.0,0.0,0.0,6.0,9.0,17.0,2.0,4.0,13.0,78.0,32.0,49.0,14.0,2.0,0.0,0.0,0,1,1,0,2016,2016-08-13,58.98,59.74,0.4482,0.2663,0.2854,1.37,1.05,36.5,29.1,1.24,1.84,1.71,1.56,0.00,1.05
4,SOUTHAMPTON,WATFORD,4.6,7.0,6.1,64,36,10.0,6.0,24.0,8.0,6.0,25.0,85.0,9.0,21.0,8.0,1.0,0.0,0.0,4.0,1.0,5.0,0.0,2.0,12.0,69.0,19.0,36.0,12.0,2.0,0.0,1.0,1,1,1,1,2016,2016-08-13,69.49,59.33,0.5759,0.1874,0.2367,1.91,1.05,34.1,30.7,1.05,0.22,1.52,0.41,1.05,1.05


In [9]:
df.shape

(1900, 54)

***IDENTIFICATION DES VALEURS MANQUANTES***


In [10]:
# df_copy = df.copy()

# # supprimer les valeurs manquantes
# df.dropna(inplace=True)

# # identifier les lignes supprimées
# deleted_rows = df_copy[~df_copy.isin(df)].dropna(how='all')
# print("Les lignes supprimées sont:")
# print(deleted_rows)


*(les valeurs manquantes proviennent uniquement des features "Importance" on peut donc les remplacer par la moyenne)*

In [11]:
# créer une copie du dataframe original
meanHTI = df["HT Importance"].mean()
meanATI = df["AT Importance"].mean()

df["HT Importance"] = df["HT Importance"].fillna(meanHTI)
df["AT Importance"] = df["AT Importance"].fillna(meanATI)


***AJOUT DE DEUX COLONNES***

In [12]:
df = df.dropna() 
df["Score Difference"] = df["HT Goals Scored"] - df["AT Goals Scored"]  # on renseigne l'ecart de score
df["Match Outcome"] = df["Score Difference"].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0)) # on renseigne l'issue du match (1/0/-1)

df['MatchWeek'] = (df.index // 10) + 1 # nouvelle colonne qui donne la "journée" du match (semaine)
col_to_move = df.pop('MatchWeek')
df.insert(0, 'MatchWeek', col_to_move)

col_to_move1 = df.pop('date')
df.insert(0, 'date', col_to_move1)
df["date"] = pd.to_datetime(df["date"]) # on convertie le type de "date"

df = df.sort_values("date") # on trie chronologiquement
df = df.round(2) # 1 chifre après la virgule

***
## **DATA VISUALIZATION :** ## 
***


In [13]:
# df.mean()
df.describe()

,MatchWeek,Match Excitement,HT Rating,AT Rating,HT Posses %,AT Posses %,HT OffT Shots,HT OnT Shots,HT TotShots,HT Block Shots,HT Corners,HT Throw Ins,HT Pass %,HT Aerials Won,HT Clear,HT Fouls,HT YellowC,HT SecYellowC,HT RedC,AT OffT Shots,AT OnT Shots,AT TotShots,AT Block Shots,AT Corners,AT Throw Ins,AT Pass %,AT Aerials Won,AT Clear,AT Fouls,AT YellowC,AT SecYellowC,AT RedC,HT Goals Scored,AT Goals Scored,HT Goals Conceeded,AT Goals Conceeded,season,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore,Score Difference,Match Outcome
count,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.0000,1900.0000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000,1900.000000
mean,95.500000,5.355579,6.421105,6.216158,51.192105,48.807895,5.223684,4.667368,13.598421,3.719474,5.691053,21.878947,78.314737,18.452632,20.655263,10.624211,1.578421,0.018947,0.032632,4.321579,3.931579,11.269474,3.032105,4.676842,21.144211,77.187368,18.419474,24.302105,10.796316,1.708421,0.027368,0.036842,1.513158,1.2300,1.2300,1.513158,2018.000000,70.213442,70.139032,0.451116,0.313368,0.235542,1.573911,1.224074,35.163916,33.967332,1.554053,1.275211,1.540632,1.273358,1.517679,1.240379,0.283158,0.127895
std,54.861955,1.675894,0.996768,0.992519,13.056100,13.056100,2.705477,2.660516,5.713661,2.548061,3.067062,6.199006,7.722811,7.444608,9.737124,3.429397,1.233061,0.136375,0.183548,2.364218,2.345725,4.932191,2.212636,2.744671,6.232484,7.762716,7.364170,10.564942,3.532753,1.264925,0.163197,0.199289,1.307545,1.2096,1.2096,1.307545,1.414586,11.907131,11.912981,0.201658,0.184723,0.051541,0.529521,0.519239,26.638662,26.022795,0.904959,0.794316,0.806322,0.676755,1.276650,1.181161,1.909989,0.870777
min,1.000000,1.700000,2.400000,1.500000,18.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,51.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,48.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,2016.000000,42.970000,43.410000,0.030000,0.010000,0.050000,0.410000,0.200000,0.000000,0.000000,0.000000,0.000000,0.050000,0.030000,0.000000,0.000000,-9.000000,-1.000000
25%,48.000000,4.100000,5.700000,5.500000,42.000000,39.000000,3.000000,3.000000,9.750000,2.000000,3.000000,18.000000,73.000000,13.000000,14.000000,8.000000,1.000000,0.000000,0.000000,3.000000,2.000000,8.000000,1.000000,3.000000,17.000000,72.000000,13.000000,17.000000,8.000000,1.000000,0.000000,0.000000,1.000000,0.0000,0.0000,1.000000,2017.000000,60.847500,60.562500,0.320000,0.187500,0.210000,1.217500,0.880000,12.675000,11.175000,0.850000,0.680000,0.970000,0.780000,1.050000,0.000000,-1.000000,-1.000000
50%,95.500000,5.300000,6.400000,6.200000,51.000000,49.000000,5.000000,4.000000,13.000000,3.000000,5.000000,21.000000,79.000000,17.500000,19.000000,10.000000,1.000000,0.000000,0.000000,4.000000,4.000000,11.000000,3.000000,4.000000,21.000000,78.000000,18.000000,23.000000,11.000000,2.000000,0.000000,0.000000,1.000000,1.0000,1.0000,1.000000,2018.000000,68.955000,68.795000,0.440000,0.290000,0.250000,1.460000,1.150000,32.600000,32.250000,1.410000,1.130000,1.420000,1.170000,1.050000,1.050000,0.000000,0.000000
75%,143.000000,6.400000,7.000000,6.800000,61.000000,58.000000,7.000000,6.000000,17.000000,5.000000,8.000000,26.000000,84.000000,23.000000,26.000000,13.000000,2.000000,0.000000,0.000000,6.0

In [14]:
df.head()

,date,MatchWeek,Home Team,Away Team,Match Excitement,HT Rating,AT Rating,HT Posses %,AT Posses %,HT OffT Shots,HT OnT Shots,HT TotShots,HT Block Shots,HT Corners,HT Throw Ins,HT Pass %,HT Aerials Won,HT Clear,HT Fouls,HT YellowC,HT SecYellowC,HT RedC,AT OffT Shots,AT OnT Shots,AT TotShots,AT Block Shots,AT Corners,AT Throw Ins,AT Pass %,AT Aerials Won,AT Clear,AT Fouls,AT YellowC,AT SecYellowC,AT RedC,HT Goals Scored,AT Goals Scored,HT Goals Conceeded,AT Goals Conceeded,season,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore,Score Difference,Match Outcome
0,2016-08-13,1,HULL CITY,LEICESTER CITY,5.9,6.7,5.4,50,50,7.0,5.0,14.0,2.0,5.0,23.0,76.0,19.0,29.0,8.0,2.0,0.0,0.0,9.0,5.0,18.0,4.0,3.0,16.0,78.0,18.0,15.0,17.0,2.0,0.0,0.0,2,1,1,2,2016,53.57,66.81,0.35,0.36,0.29,1.16,1.24,38.1,22.2,0.85,2.77,0.17,1.25,2.10,1.05,1,1
1,2016-08-13,1,EVERTON,TOTTENHAM,4.9,7.0,6.4,42,58,4.0,6.0,12.0,2.0,5.0,21.0,73.0,16.0,40.0,10.0,0.0,0.0,0.0,4.0,4.0,13.0,5.0,6.0,42.0,80.0,10.0,22.0,14.0,0.0,0.0,0.0,1,1,1,1,2016,68.02,73.25,0.39,0.34,0.27,1.47,1.38,31.9,48.0,0.73,1.11,0.88,1.81,1.05,1.05,0,0
2,2016-08-13,1,MIDDLESBROUGH,STOKE,4.5,6.4,6.2,46,54,6.0,2.0,12.0,4.0,9.0,20.0,75.0,22.0,20.0,18.0,3.0,0.0,0.0,9.0,1.0,12.0,2.0,6.0,23.0,79.0,27.0,16.0,14.0,5.0,0.0,0.0,1,1,1,1,2016,56.32,60.35,0.44,0.27,0.29,1.30,1.01,33.9,32.5,1.40,0.55,1.13,1.06,1.05,1.05,0,0
3,2016-08-13,1,BURNLEY,SWANSEA,4.0,5.1,7.9,47,53,5.0,3.0,10.0,2.0,7.0,41.0,71.0,20.0,16.0,10.0,3.0,0.0,0.0,6.0,9.0,17.0,2.0,4.0,13.0,78.0,32.0,49.0,14.0,2.0,0.0,0.0,0,1,1,0,2016,58.98,59.74,0.45,0.27,0.29,1.37,1.05,36.5,29.1,1.24,1.84,1.71,1.56,0.00,1.05,-1,-1
4,2016-08-13,1,SOUTHAMPTON,WATFORD,4.6,7.0,6.1,64,36,10.0,6.0,24.0,8.0,6.0,25.0,85.0,9.0,21.0,8.0,1.0,0.0,0.0,4.0,1.0,5.0,0.0,2.0,12.0,69.0,19.0,36.0,12.0,2.0,0.0,1.0,1,1,1,1,2016,69.49,59.33,0.58,0.19,0.24,1.91,1.05,34.1,30.7,1.05,0.22,1.52,0.41,1.05,1.05,0,0


---
---
---

***
### **ENCODAGE :** ###
***

In [15]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(df[["Home Team", "Away Team"]]) # encodeur sur les noms des équipes
df[["Home Team", "Away Team"]] = encoder.transform(df[["Home Team", "Away Team"]]) # transformation des noms des équipes en nombres
df[["Home Team", "Away Team"]].head()

,Home Team,Away Team
0,11.0,13.0
1,8.0,25.0
2,17.0,22.0
3,4.0,24.0
4,21.0,26.0


In [16]:
def decode():
    df[["Home Team", "Away Team"]] = encoder.inverse_transform(df[["Home Team", "Away Team"]]) # transformation des nombres en noms des équipes e

def encode():
    df[["Home Team", "Away Team"]] = encoder.transform(df[["Home Team", "Away Team"]])# transformation des noms des équipes en nombres

In [17]:
decode()

In [18]:
df.head()

,date,MatchWeek,Home Team,Away Team,Match Excitement,HT Rating,AT Rating,HT Posses %,AT Posses %,HT OffT Shots,HT OnT Shots,HT TotShots,HT Block Shots,HT Corners,HT Throw Ins,HT Pass %,HT Aerials Won,HT Clear,HT Fouls,HT YellowC,HT SecYellowC,HT RedC,AT OffT Shots,AT OnT Shots,AT TotShots,AT Block Shots,AT Corners,AT Throw Ins,AT Pass %,AT Aerials Won,AT Clear,AT Fouls,AT YellowC,AT SecYellowC,AT RedC,HT Goals Scored,AT Goals Scored,HT Goals Conceeded,AT Goals Conceeded,season,HT SPI Rate,AT SPI Rate,HT ProbWin,AT ProbWin,probtie,HT ProjScore,AT ProjScore,HT Importance,AT Importance,HT xG,AT xG,HT NSxG,AT NSxG,HT AdjScore,AT AdjScore,Score Difference,Match Outcome
0,2016-08-13,1,HULL CITY,LEICESTER CITY,5.9,6.7,5.4,50,50,7.0,5.0,14.0,2.0,5.0,23.0,76.0,19.0,29.0,8.0,2.0,0.0,0.0,9.0,5.0,18.0,4.0,3.0,16.0,78.0,18.0,15.0,17.0,2.0,0.0,0.0,2,1,1,2,2016,53.57,66.81,0.35,0.36,0.29,1.16,1.24,38.1,22.2,0.85,2.77,0.17,1.25,2.10,1.05,1,1
1,2016-08-13,1,EVERTON,TOTTENHAM,4.9,7.0,6.4,42,58,4.0,6.0,12.0,2.0,5.0,21.0,73.0,16.0,40.0,10.0,0.0,0.0,0.0,4.0,4.0,13.0,5.0,6.0,42.0,80.0,10.0,22.0,14.0,0.0,0.0,0.0,1,1,1,1,2016,68.02,73.25,0.39,0.34,0.27,1.47,1.38,31.9,48.0,0.73,1.11,0.88,1.81,1.05,1.05,0,0
2,2016-08-13,1,MIDDLESBROUGH,STOKE,4.5,6.4,6.2,46,54,6.0,2.0,12.0,4.0,9.0,20.0,75.0,22.0,20.0,18.0,3.0,0.0,0.0,9.0,1.0,12.0,2.0,6.0,23.0,79.0,27.0,16.0,14.0,5.0,0.0,0.0,1,1,1,1,2016,56.32,60.35,0.44,0.27,0.29,1.30,1.01,33.9,32.5,1.40,0.55,1.13,1.06,1.05,1.05,0,0
3,2016-08-13,1,BURNLEY,SWANSEA,4.0,5.1,7.9,47,53,5.0,3.0,10.0,2.0,7.0,41.0,71.0,20.0,16.0,10.0,3.0,0.0,0.0,6.0,9.0,17.0,2.0,4.0,13.0,78.0,32.0,49.0,14.0,2.0,0.0,0.0,0,1,1,0,2016,58.98,59.74,0.45,0.27,0.29,1.37,1.05,36.5,29.1,1.24,1.84,1.71,1.56,0.00,1.05,-1,-1
4,2016-08-13,1,SOUTHAMPTON,WATFORD,4.6,7.0,6.1,64,36,10.0,6.0,24.0,8.0,6.0,25.0,85.0,9.0,21.0,8.0,1.0,0.0,0.0,4.0,1.0,5.0,0.0,2.0,12.0,69.0,19.0,36.0,12.0,2.0,0.0,1.0,1,1,1,1,2016,69.49,59.33,0.58,0.19,0.24,1.91,1.05,34.1,30.7,1.05,0.22,1.52,0.41,1.05,1.05,0,0


In [19]:
#on copie de df pour faire un autre pour chaque équipe
df_matches = df
df_teams = pd.DataFrame()

for team in df_matches["Home Team"].unique(): # on boucle sur chaque équipe unique 

    df_current_team = pd.DataFrame()  # on prend un nouveau DataFrame pour l'équipe courante de la boucle 

    df_home_matches = df_matches[df_matches["Home Team"] == team] # match à domicile de l'équipe courante
    df_away_matches = df_matches[df_matches["Away Team"] == team] # match à l'exterieur de l'équipe courante
    df_all_matches = pd.concat([df_home_matches, df_away_matches]) # on assemble pour avoir tous les matchs de l'équipe courante

    df_current_team["Team"] = [team]
    # on crée une colonne average pour chaque features (*)
    average_possession = (df_home_matches["HT Posses %"].mean() + df_away_matches["AT Posses %"].mean()) / 2
    df_current_team["Average Posses"] = average_possession

    # Features à calculer la moyenne pour chaque équipe
    featuresHA = ["Goals Scored", "Goals Conceeded", "xG", "NSxG", "TotShots", "OffT Shots", "OnT Shots", "Block Shots", "Pass %", 
            "Corners", "Throw Ins", "Aerials Won", "Clear", 
            "Fouls", "YellowC", "SecYellowC", "RedC", "Importance", "ProjScore", "AdjScore", "SPI Rate", "ProbWin"]

    # on repete le processus (*) pour chaque feature de type ("HT" ou "Away Team")
    for feature in featuresHA:
        home_feature_mean = df_home_matches["HT " + feature].mean()  # moyenne pour les matchs à domicile
        away_feature_mean = df_away_matches["AT " + feature].mean()  # moyenne pour les matchs à l'extérieur

        avg_feature = (home_feature_mean + away_feature_mean) / 2 # moyenne pour tous les matchs
        df_current_team["Average " + feature] = avg_feature

    # on fait la même chose pour les issues de match et la différence de buts
    features = ["Match Outcome", "Score Difference"]
    
    for feature in features:

        home_feature_mean = df_home_matches[feature].mean()  # moyenne pour les matchs à domicile
        away_feature_mean = df_away_matches[feature].mean()  # moyenne pour les matchs à l'extérieur

        avg_feature = (home_feature_mean - away_feature_mean) / 2 # moyenne pour tous les matchs (on fait bien - pour inverser les -1 des victoires à l'extérieur)
        df_current_team["Average " + feature] = avg_feature
      
    df_teams = pd.concat([df_teams, df_current_team], ignore_index=True) # on ajoute la ligne pour l'équipe courante au DF de toutes les équipes


In [20]:
df_teams.sort_values("Team").head()

,Team,Average Posses,Average Goals Scored,Average Goals Conceeded,Average xG,Average NSxG,Average TotShots,Average OffT Shots,Average OnT Shots,Average Block Shots,Average Pass %,Average Corners,Average Throw Ins,Average Aerials Won,Average Clear,Average Fouls,Average YellowC,Average SecYellowC,Average RedC,Average Importance,Average ProjScore,Average AdjScore,Average SPI Rate,Average ProbWin,Average Match Outcome,Average Score Difference
8,ARSENAL,57.347368,1.763158,1.226316,1.645000,1.897842,13.121053,4.784211,4.778947,3.568421,83.200000,5.784211,19.878947,14.963158,19.589474,10.305263,1.757895,0.015789,0.073684,44.047526,1.808053,1.756789,80.288789,0.503368,0.210526,0.536842
27,ASTON VILLA,45.973684,1.263158,1.486842,1.388816,1.278684,12.894737,4.934211,4.447368,3.539474,76.697368,5.434211,20.631579,18.592105,20.500000,11.710526,1.789474,0.065789,0.000000,37.890263,1.240789,1.276316,68.969868,0.313289,-0.144737,-0.223684
7,BOURNEMOUTH,47.532895,1.289474,1.730263,1.313158,1.301974,11.519737,4.394737,3.960526,3.177632,76.782895,5.236842,21.736842,18.197368,26.776316,9.315789,1.618421,0.032895,0.019737,22.522961,1.244013,1.295132,62.325000,0.297237,-0.184211,-0.440789
20,BRIGHTON,47.276316,0.973684,1.407895,1.191382,1.207303,11.197368,4.730263,3.361842,3.105263,77.000000,4.710526,21.671053,18.782895,21.453947,11.105263,1.447368,0.026316,0.065789,25.067763,1.063355,0.998618,63.984276,0.281316,-0.197368,-0.434211
3,BURNLEY,41.489474,1.031579,1.405263,1.120158,1.163737,10.010526,4.215789,3.310526,2.500000,69.189474,4.205263,21.636842,24.889474,26.752632,10.173684,1.689474,0.005263,0.010526,21.561789,1.075947,1.046684,63.304368,0.283000,-0.126316,-0.373684


In [21]:
# df_matchs = df[["Home Team","Away Team", "Match Outcome"]]
# confront = matchs[(matchs['Home Team'] == 15.0)|(matchs['Away Team'] == 15.0)]
# df_matchs.head()

In [22]:
# Fonction pour calculer la valeur moyenne d'un feature lors des 3 derniers matchs pour une équipe (feature de type "HT ..." ou "AT ....")
def get_HA_L3M(df, team, date, match_id, feature):
    previous_matches = df[(df['Home Team'] == team) | (df['Away Team'] == team)]
    previous_matches = previous_matches[previous_matches.index < match_id].sort_values('date', ascending=False).iloc[:3]
    f_score = 0
    for index, row in previous_matches.iterrows():
        if row['Home Team'] == team:
            f_score += row["HT "+ feature]
        else:
            f_score += row["AT "+ feature]
    return f_score/3

# Fonction pour calculer la valeur moyenne d'un feature lors des 3 derniers matchs pour une équipe (feature de type "HT ..." ou "AT ....")
def get_L3M(df, team, date, match_id, feature):
    previous_matches = df[(df['Home Team'] == team) | (df['Away Team'] == team)]
    previous_matches = previous_matches[previous_matches.index < match_id].sort_values('date', ascending=False).iloc[:3]
    f_score = 0
    for index, row in previous_matches.iterrows():
        if row['Home Team'] == team:
            f_score += row[feature]
        else:
            f_score -= row[feature] # (on fait bien - pour inverser les -1 des victoires à l'extérieur) ou la score difference
    return f_score/3



In [23]:
featuresL3M = ["Goals Scored", "Goals Conceeded", "xG", "NSxG", "TotShots",
                "RedC", "Importance"]
fL3M = ["Match Outcome", "Score Difference"]

# on applque les fonctions à tous les features 

for feature in featuresL3M:
    df["HT " + feature + " L3M"] = df.apply(lambda x: get_HA_L3M(df, x['Home Team'], x['date'], x.name, feature), axis=1)
    df["AT " + feature + " L3M"] = df.apply(lambda x: get_HA_L3M(df, x['Away Team'], x['date'], x.name, feature), axis=1)
for feature in fL3M:
    df["HT " + feature + " L3M"] = df.apply(lambda x: get_L3M(df, x['Home Team'], x['date'], x.name, feature), axis=1)
    df["AT " + feature + " L3M"] = df.apply(lambda x: get_L3M(df, x['Away Team'], x['date'], x.name, feature), axis=1)
    

########################################################
# ATTENTION 38 SECONDES POUR L'EXECUTION DE LA CELLULE #
########################################################
    

In [24]:
# vv = df_with_L3M_GS.groupby(['Home Team', 'Away Team'])
# group = vv.get_group(("LIVERPOOL", "MANCHESTER CITY"))
# # group.head


In [25]:
df = df.round(2)



In [26]:
liverpool_matches = df[(df["Home Team"]== "LIVERPOOL") | (df["Away Team"]== "LIVERPOOL") ]
liverpool_matches[["MatchWeek","Home Team", "Away Team", "Match Outcome", "HT Match Outcome L3M", "AT Match Outcome L3M","Score Difference", "HT Score Difference L3M", "AT Score Difference L3M"]].head(10)

,MatchWeek,Home Team,Away Team,Match Outcome,HT Match Outcome L3M,AT Match Outcome L3M,Score Difference,HT Score Difference L3M,AT Score Difference L3M
8,1,ARSENAL,LIVERPOOL,-1,0.00,0.00,-1,0.00,0.00
15,2,BURNLEY,LIVERPOOL,1,-0.33,0.33,2,-0.33,0.33
20,3,TOTTENHAM,LIVERPOOL,0,0.33,0.00,0,0.33,-0.33
37,4,LIVERPOOL,LEICESTER CITY,1,0.00,0.00,3,-0.33,0.00
40,5,CHELSEA,LIVERPOOL,-1,0.67,0.00,-1,1.33,0.33
51,6,LIVERPOOL,HULL CITY,1,0.67,-0.67,4,1.33,-1.33
61,7,SWANSEA,LIVERPOOL,-1,-0.67,1.00,-1,-1.00,2.67
79,8,LIVERPOOL,MANCHESTER UNITED,0,1.00,0.00,0,2.00,0.33
87,9,LIVERPOOL,WEST BROM,1,0.67,0.00,1,1.67,0.00
96,10,CRYSTAL PALACE,LIVERPOOL,-1,-0.67,0.67,-2,-1.00,0.67


In [29]:
features = ['date', 'MatchWeek', 'Home Team', 'Away Team', 'Match Excitement',
       'HT Rating', 'AT Rating', 'HT Posses %', 'AT Posses %', 'HT OffT Shots',
       'HT OnT Shots', 'HT TotShots', 'HT Block Shots', 'HT Corners',
       'HT Throw Ins', 'HT Pass %', 'HT Aerials Won', 'HT Clear', 'HT Fouls',
       'HT YellowC', 'HT SecYellowC', 'HT RedC', 'AT OffT Shots',
       'AT OnT Shots', 'AT TotShots', 'AT Block Shots', 'AT Corners',
       'AT Throw Ins', 'AT Pass %', 'AT Aerials Won', 'AT Clear', 'AT Fouls',
       'AT YellowC', 'AT SecYellowC', 'AT RedC', 'HT Goals Scored',
       'AT Goals Scored', 'HT Goals Conceeded', 'AT Goals Conceeded', 'season',
       'HT SPI Rate', 'AT SPI Rate', 'HT ProbWin', 'AT ProbWin', 'probtie',
       'HT ProjScore', 'AT ProjScore', 'HT Importance', 'AT Importance',
       'HT xG', 'AT xG', 'HT NSxG', 'AT NSxG', 'HT AdjScore', 'AT AdjScore',
       'Score Difference', 'Match Outcome', 'HT Goals Scored L3M',
       'AT Goals Scored L3M', 'HT Goals Conceeded L3M',
       'AT Goals Conceeded L3M', 'HT xG L3M', 'AT xG L3M', 'HT NSxG L3M',
       'AT NSxG L3M', 'HT TotShots L3M', 'AT TotShots L3M', 'HT RedC L3M',
       'AT RedC L3M', 'HT Importance L3M', 'AT Importance L3M',
       'HT Match Outcome L3M', 'AT Match Outcome L3M',
       'HT Score Difference L3M', 'AT Score Difference L3M']


In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df[features], df["Match Outcome"], test_size=0.3, random_state=42)

# Entraînement d'un modèle de forêt aléatoire
clf = RandomForestClassifier(n_estimators=50, random_state=42, min_samples_split=5, min_samples_leaf=1, max_features="sqrt", max_depth=20, bootstrap=True )
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)
y_pred2 = clf.predict_proba(X_test)

# scores = cross_val_score(clf, df[features], df["Score Difference"], cv=5)
# print("Cross-validation scores:", scores*100)
# print("Average score:", scores.mean()*100)

# Calcul de la précision
accuracy = accuracy_score(y_test, y_pred)
print("Précision : {:.2f}%".format(accuracy * 100))


TypeError: float() argument must be a string or a number, not 'Timestamp'

In [ ]:
df_teams

,Team,Average Posses,Average Goals Scored,Average Goals Conceeded,Average xG,Average NSxG,Average TotShots,Average OffT Shots,Average OnT Shots,Average Block Shots,Average Pass %,Average Corners,Average Throw Ins,Average Aerials Won,Average Clear,Average Fouls,Average YellowC,Average SecYellowC,Average RedC,Average Importance,Average ProjScore,Average AdjScore,Average SPI Rate,Average ProbWin,Average Match Outcome,Average Score Difference
0,HULL CITY,45.921053,0.973684,2.105263,0.908684,0.930263,10.447368,4.815789,3.315789,2.315789,76.500000,4.710526,19.000000,15.473684,28.973684,10.578947,1.763158,0.000000,0.131579,47.326316,0.900263,0.993421,47.601842,0.208684,-0.342105,-1.131579
1,EVERTON,49.000000,1.321053,1.326316,1.358526,1.290579,11.731579,4.473684,4.168421,3.110526,77.121053,4.900000,21.578947,20.131579,24.552632,11.631579,1.652632,0.036842,0.036842,15.340474,1.314316,1.322684,70.406684,0.357579,0.036842,-0.005263
2,MIDDLESBROUGH,47.236842,0.710526,1.394737,0.908684,1.015263,9.236842,3.947368,2.631579,2.657895,76.184211,3.710526,19.631579,20.368421,28.684211,12.684211,2.052632,0.026316,0.000000,35.102368,0.930263,0.746053,53.008947,0.257105,-0.394737,-0.684211
3,BURNLEY,41.489474,1.031579,1.405263,1.120158,1.163737,10.010526,4.215789,3.310526,2.500000,69.189474,4.205263,21.636842,24.889474,26.752632,10.173684,1.689474,0.005263,0.010526,21.561789,1.075947,1.046684,63.304368,0.283000,-0.126316,-0.373684
4,SOUTHAMPTON,50.142105,1.163158,1.563158,1.321895,1.232316,12.663158,5.005263,4.436842,3.268421,75.868421,5.289474,22.357895,18.142105,22.989474,11.263158,1.594737,0.026316,0.047368,22.527474,1.222579,1.176737,67.221474,0.323211,-0.152632,-0.400000
5,CRYSTAL PALACE,43.994737,1.147368,1.510526,1.182368,1.265105,11.226316,4.515789,3.621053,3.110526,74.410526,4.978947,22.115789,19.526316,23.600000,11.126316,1.715789,0.010526,0.021053,21.140474,1.135526,1.144263,64.624316,0.291684,-0.152632,-0.363158
6,MANCHESTER CITY,67.131579,2.452632,0.821053,2.396895,2.429053,17.505263,6.273684,6.447368,4.784211,88.105263,7.405263,19.278947,13.894737,15.452632,9.415789,1.494737,0.021053,0.047368,52.783368,2.423737,2.433632,91.530053,0.711737,0.594737,1.631579
7,BOURNEMOUTH,47.532895,1.289474,1.730263,1.313158,1.301974,11.519737,4.394737,3.960526,3.177632,76.782895,5.236842,21.736842,18.197368,26.776316,9.315789,1.618421,0.032895,0.019737,22.522961,1.244013,1.295132,62.325000,0.297237,-0.184211,-0.440789
8,ARSENAL,57.347368,1.763158,1.226316,1.645000,1.897842,13.121053,4.784211,4.778947,3.568421,83.200000,5.784211,19.878947,14.963158,19.589474,10.305263,1.757895,0.015789,0.073684,44.047526,1.808053,1.756789,80.288789,0.503368,0.210526,0.536842
9,CHELSEA,59.215789,1.773684,1.052632,1.783211,1.916211,15.663158,5.584211,5.542105,4.536842,85.047368,5.978947,20.500000,15.473684,17.736842,10.084211,1.447368,0.015789,0.021053,64.628211,1.929211,1.772842,84.988000,0.585316,0.352632,0.721053


In [ ]:
teams_ID.sort_values("Team ID")

,Team ID,Team Name
8,0.0,ARSENAL
27,1.0,ASTON VILLA
7,2.0,BOURNEMOUTH
20,3.0,BRIGHTON
3,4.0,BURNLEY
25,5.0,CARDIFF
9,6.0,CHELSEA
5,7.0,CRYSTAL PALACE
1,8.0,EVERTON
23,9.0,FULHAM


In [74]:
home_matches = df[["season","date","Home Team", "Away Team", "HT Rating", "HT Posses %", "HT OffT Shots", "HT OnT Shots", "HT TotShots", "HT Block Shots", "HT Corners", "HT Throw Ins", "HT Pass %", "HT Aerials Won", "HT Clear", "HT Fouls", "HT YellowC", "HT SecYellowC", "HT RedC", "HT xG","HT NSxG","HT Goals Scored", "HT Goals Conceeded", "Score Difference", "Match Outcome"]]

away_matches = df[["season","date","Away Team", "Home Team", "AT Rating", "AT Posses %", "AT OffT Shots", "AT OnT Shots", "AT TotShots", "AT Block Shots", "AT Corners", "AT Throw Ins", "AT Pass %", "AT Aerials Won", "AT Clear", "AT Fouls", "AT YellowC", "AT SecYellowC", "AT RedC", "AT xG","AT NSxG","AT Goals Scored", "AT Goals Conceeded","Score Difference", "Match Outcome"]]



In [87]:
def prepare_data(df_home, df_away, n_matches):
    data = []
    for _, row in df_home.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        date = row["date"]
        previous_home = df_home[(df_home["Home Team"] == home_team) & (df_home["date"] < date)].iloc[-n_matches:, 5:].values
        previous_away = df_away[(df_away["Away Team"] == away_team) & (df_away["date"] < date)].iloc[-n_matches:, 5:].values
        if len(previous_home) == n_matches and len(previous_away) == n_matches:
            home_stats = previous_home.flatten().tolist()
            away_stats = previous_away.flatten().tolist()
            outcome = row["Match Outcome"]
            data.append((home_stats + away_stats, outcome))
    return pd.DataFrame(data)


In [88]:
data = prepare_data(home_matches, away_matches, 3)

In [109]:
len(data[0][0])

120

In [77]:
def create_match_dataset(df):
    match_dataset = []
    for index, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        date = row["date"]
        home_team_stats = df[(df["Home Team"] == home_team) & (df["date"] < date)].iloc[:, 5:].values.tolist()
        away_team_stats = df[(df["Away Team"] == away_team) & (df["date"] < date)].iloc[:, 5:].values.tolist()
        if len(home_team_stats) > 0 and len(away_team_stats) > 0:
            home_last_stats = home_team_stats[-1][:-1]
            away_last_stats = away_team_stats[-1][:-1]
            match_outcome = row["Match Outcome"]
            match_dataset.append(home_last_stats + away_last_stats + [match_outcome])
    return pd.DataFrame(match_dataset)


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

# Importer les données
match_df = matchs 
confrontations_df = df_teams

# Créer un ensemble de données de caractéristiques et un vecteur de cibles
X = match_df.drop(columns=["Match Outcome"])
y = match_df["Match Outcome"]

# Diviser l'ensemble de données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instancier le modèle de régression logistique et l'ajuster aux données d'entraînement
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Utiliser la validation croisée pour évaluer la performance du modèle
scores = cross_val_score(lr, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average score:", scores.mean())

# Utiliser le modèle pour faire des prédictions sur les confrontations
confrontations_X = confrontations_df.drop(columns=["Match Outcome"])
confrontations_y_pred = lr.predict(confrontations_X)

# Afficher les prédictions
print("Predictions for upcoming matches:")
for i, row in confrontations_df.iterrows():
    print(row["Home Team"], "vs", row["Away Team"], ":", confrontations_y_pred[i])
